In [3]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter

In [5]:
raw = pd.read_csv( "data/data_clean_clean.csv")
slang_words = pd.read_csv("data/external/kbba_ID.txt",
                        sep="\t", header=None)
slang = pd.read_csv("data/external/slangword_ID.txt",
                        sep=":", header=None)
baku_words = pd.read_csv("data/external/katabaku_ID.txt",
                        sep="|", header=None)
baku_words.columns = [1,0]
root_words = np.array(pd.read_csv("data/external/rootword_ID.txt",
                        sep="\n", header=None).values)
slang_words = pd.concat([slang_words, slang, baku_words])

In [6]:
raw

,review,polarity
0,"'membantu', 'kondisi', 'pandemi', 'tolong', 'e...",negatif
1,"'konsultasi', 'dokter', 'bagus', 'durasinya', ...",negatif
2,"'astaga', 'obatnya', 'beli', 'obat', 'harganya...",negatif
3,"'terima kasih', 'pelayananya', 'baik', 'cepat'...",positif
4,"'menelepon', 'nomor', 'hadiah', 'konfirmasi', ...",positif
...,...,...
795,"'semudah', 'diklik', 'praktis', 'bantu'",positif
796,"'pelayananya', 'ramah', 'paham', 'bidangnya'",positif
797,"'bagus', 'aplikasi', 'dokter', 'pribadi', 'ter...",positif
798,"'memudahkan', 'konsultasi', 'rumah'",positif


In [7]:
slang_words.drop_duplicates(inplace=True)
slang_words = dict(zip(slang_words[0],slang_words[1]))
raw['review'] = raw['review'].apply(lambda x: x.replace(","," "))
def delete_suffix_nya(review):
    return re.sub("(?:nya|ny|y)[$|\s]"," ",review)
    
raw['review'] = raw['review'].apply(delete_suffix_nya)
raw['review'] = raw['review'].apply(lambda x: x.split())

In [8]:
raw

,review,polarity
0,"['membantu', 'kondisi', 'pandemi', 'tolong', '...",negatif
1,"['konsultasi', 'dokter', 'bagus', 'durasinya',...",negatif
2,"['astaga', 'obatnya', 'beli', 'obat', 'hargany...",negatif
3,"['terima, kasih', 'pelayananya', 'baik', 'cepa...",positif
4,"['menelepon', 'nomor', 'hadiah', 'konfirmasi',...",positif
...,...,...
795,"['semudah', 'diklik', 'praktis', 'bantu']",positif
796,"['pelayananya', 'ramah', 'paham', 'bidangnya']",positif
797,"['bagus', 'aplikasi', 'dokter', 'pribadi', 'te...",positif
798,"['memudahkan', 'konsultasi', 'rumah']",positif


In [11]:
def mapping_slang_words(review):
    return [slang_words[word] if word in slang_words else word for word in review]
raw['review'] = raw['review'].apply(mapping_slang_words)

In [12]:
def convert_list_to_string(word_list):
    return ",".join(word_list)
raw['review'] = raw['review'].apply(convert_list_to_string)
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]

In [13]:
def avg_word(words):
  return (sum(len(word) for word in words)/len(words))

raw['avg_word'] = raw['review'].apply(lambda x: avg_word(x.split(",")))

In [14]:
raw.head()

,review,polarity,avg_word
0,"'membantu','kondisi','pandemi','tolong','evalu...",negatif,8.045455
1,"'konsultasi','dokter','bagus','durasinya','cep...",negatif,10.000000
2,"'astaga','obatnya','beli','obat','harganya','m...",negatif,7.766667
3,"'terima,kasih','pelayananya','baik','cepat','p...",positif,9.000000
4,"'menelepon','nomor','hadiah','konfirmasi','apl...",positif,8.909091


In [15]:
raw['review'] = raw['review'].astype(str)
raw.head()

,review,polarity,avg_word
0,"'membantu','kondisi','pandemi','tolong','evalu...",negatif,8.045455
1,"'konsultasi','dokter','bagus','durasinya','cep...",negatif,10.000000
2,"'astaga','obatnya','beli','obat','harganya','m...",negatif,7.766667
3,"'terima,kasih','pelayananya','baik','cepat','p...",positif,9.000000
4,"'menelepon','nomor','hadiah','konfirmasi','apl...",positif,8.909091


In [16]:
raw['review'] = raw['review'].apply(lambda x: x.strip())
raw.head()

,review,polarity,avg_word
0,"'membantu','kondisi','pandemi','tolong','evalu...",negatif,8.045455
1,"'konsultasi','dokter','bagus','durasinya','cep...",negatif,10.000000
2,"'astaga','obatnya','beli','obat','harganya','m...",negatif,7.766667
3,"'terima,kasih','pelayananya','baik','cepat','p...",positif,9.000000
4,"'menelepon','nomor','hadiah','konfirmasi','apl...",positif,8.909091


In [18]:
raw.head()

,review,polarity,avg_word
0,"'membantu','kondisi','pandemi','tolong','evalu...",negatif,8.045455
1,"'konsultasi','dokter','bagus','durasinya','cep...",negatif,10.000000
2,"'astaga','obatnya','beli','obat','harganya','m...",negatif,7.766667
3,"'terima,kasih','pelayananya','baik','cepat','p...",positif,9.000000
4,"'menelepon','nomor','hadiah','konfirmasi','apl...",positif,8.909091


In [20]:
raw.to_csv("data/1.2_data_cleaner_pure.csv", index=False)